In [3]:
import sys
sys.path
sys.path.insert(0, '/Users/zhengzezhou/scikit-learn')

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot
from statsmodels.graphics.gofplots import qqplot
from scipy import stats
from tqdm import tqdm

In [5]:
def gen(n):

    x1 = np.random.uniform(size = n).reshape((n, 1))
    x2 = np.random.uniform(size = n).reshape((n, 1))
    x3 = np.random.uniform(size = n).reshape((n, 1))
    x4 = np.random.uniform(size = n).reshape((n, 1))
    x5 = np.random.uniform(size = n).reshape((n, 1))

    X = np.concatenate((x1, x2, x3, x4, x5), axis = 1)

    epi = np.random.randn(n).reshape((n, 1))

    y = 10 * np.sin(np.pi * x1 * x2) + 20 * (x3 - 0.05) ** 2 + 10 * x4 + 5 * x5 + epi

    y = y.ravel()
    
    return X, y

In [7]:
def predict_var(self, X, method = "corrected"):

    predict_all = np.zeros((X.shape[0], self.n_estimators))
    for t_idx in range(self.n_estimators):
        predict_all[:, t_idx] = self.estimators_[t_idx].predict(X)
    pred = np.mean(predict_all, axis = 1)

    inbag_times_ = self.inbag_times_

    m = X.shape[0]

    if method == "BM":
        cond_exp_full = np.zeros((self.n_samples_, m))

        for i in range(self.n_samples_):
            # cond_exp_full[i, :] = np.mean(predict_all[:, inbag_times_[i, :] >= 1], axis=1)
            cond_exp_full[i, :] = np.average(predict_all, weights = inbag_times_[i, :], axis = 1)

        zeta1_full = np.zeros(m)
        zetan_full = np.zeros(m)
        variance = np.zeros(m)
        for i in range(m):
            zeta1_full[i] = np.var(cond_exp_full[:, i])
            zetan_full[i] = np.var(predict_all[i, :])
        variance = zeta1_full * (self.n_subsamples_ ** 2) / self.n_samples_ + zetan_full / self.n_estimators
        
        return [float(variance), float(zeta1_full), float(zetan_full)]
        
    elif method == "IJ":
        
        f_centered = predict_all - np.mean(predict_all, axis=1).reshape(m, 1)
        i_centered = inbag_times_ - np.mean(inbag_times_, axis=1).reshape(self.n_samples_, 1)
        corr = np.dot(f_centered, i_centered.T) / self.n_estimators
        cov = np.dot(corr, corr.T)
        zetan_full = np.cov(predict_all)
        covariance = cov + zetan_full / self.n_estimators

        return [float(np.diagonal(covariance)), float(cov), float(zetan_full)]
    
    elif method == "corrected":
        
        cond_exp_full = np.zeros((self.n_samples_, m))

        for i in range(self.n_samples_):
            # cond_exp_full[i, :] = np.mean(predict_all[:, inbag_times_[i, :] >= 1], axis=1)
            cond_exp_full[i, :] = np.average(predict_all, weights = inbag_times_[i, :], axis = 1)

        inbag_times_ = inbag_times_[np.sum(inbag_times_, axis = 1) > 0, ]

        nk = np.sum(inbag_times_, axis = 1)
        K = len(nk)
        C = np.sum(nk)     

        SSr = np.dot(((cond_exp_full - pred) ** 2).T, nk)

        SSe = [0] * m

        for i in range(K):
            SSei = np.sum((predict_all[:, inbag_times_[i, :]>=1].T - cond_exp_full[i, :]) ** 2, axis = 0)
            SSe += SSei

        sigma_e_squared = SSe / (C - K)

        sigma_M_squared = (SSr - (K - 1) * sigma_e_squared) / (C - np.sum(nk ** 2) / C)
        
        sigma_M_squared_without = SSr / C
        
        return  [(self.n_subsamples_ ** 2) / self.n_samples_ * float(sigma_M_squared), (self.n_subsamples_ ** 2) / self.n_samples_ * float(sigma_M_squared_without)]
        

### test point 1

In [8]:
np.random.seed(11)
test = np.array([[0.5, 0.5, 0.5, 0.5, 0.5]])

rep = 500
n_sub = 100


print("========================= B = 500 =========================")

pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 1000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 1000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 2500 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 2500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


print("========================= B = 5000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 5000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


  0%|          | 0/500 [00:00<?, ?it/s]

========================= B = 500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=2.4464340380424505, pvalue=0.2942819319241859)
variance ratio for uc: 5.661924847462699
C.C. for uc: 1.0
========================= B = 1000 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=4.001477650267804, pvalue=0.135235331055992)
variance ratio for uc: 3.528958760985348
C.C. for uc: 0.998
========================= B = 2500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=1.6279248088272402, pvalue=0.44309884632997976)
variance ratio for uc: 1.672452389597524
C.C. for uc: 0.982
========================= B = 5000 =========================


100%|██████████| 500/500 [53:02<00:00,  6.36s/it]

normal test: NormaltestResult(statistic=3.855318739522874, pvalue=0.14548833465728636)
variance ratio for uc: 1.239591361780564
C.C. for uc: 0.966


In [9]:
np.random.seed(11)
test = np.array([[0.5, 0.5, 0.5, 0.5, 0.5]])

rep = 500
n_sub = 250


print("========================= B = 500 =========================")

pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 1000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 1000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 2500 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 2500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


print("========================= B = 5000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 5000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


  0%|          | 0/500 [00:00<?, ?it/s]

========================= B = 500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=3.8301459300000062, pvalue=0.14733108218155325)
variance ratio for uc: 3.297244062756296
C.C. for uc: 0.996
========================= B = 1000 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=5.974908225357875, pvalue=0.05041562597961733)
variance ratio for uc: 1.9543511415525263
C.C. for uc: 0.984
========================= B = 2500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=7.463131955648999, pvalue=0.023955292991145995)
variance ratio for uc: 0.9110164164914092
C.C. for uc: 0.918
========================= B = 5000 =========================


100%|██████████| 500/500 [1:09:41<00:00,  8.36s/it]

normal test: NormaltestResult(statistic=6.261067621616572, pvalue=0.04369446644773493)
variance ratio for uc: 0.5917853096746519
C.C. for uc: 0.854


In [10]:
np.random.seed(11)
test = np.array([[0.5, 0.5, 0.5, 0.5, 0.5]])

rep = 500
n_sub = 500


print("========================= B = 500 =========================")

pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 1000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 1000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 2500 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 2500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


print("========================= B = 5000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 5000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


  0%|          | 0/500 [00:00<?, ?it/s]

========================= B = 500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=28.941282481150466, pvalue=5.193741849773757e-07)
variance ratio for uc: 0.00012272103766959916
C.C. for uc: 0.014
========================= B = 1000 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=11.338814094398092, pvalue=0.0034499103037184133)
variance ratio for uc: 9.037086904509937e-05
C.C. for uc: 0.012
========================= B = 2500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=2.916084193098453, pvalue=0.23269141634972676)
variance ratio for uc: 3.1703047969792436e-05
C.C. for uc: 0.006
========================= B = 5000 =========================


100%|██████████| 500/500 [1:43:40<00:00, 12.44s/it]

normal test: NormaltestResult(statistic=5.256400703928276, pvalue=0.07220829489326178)
variance ratio for uc: 1.6362695671505015e-05
C.C. for uc: 0.004


### test point 2

In [11]:
np.random.seed(11)

x1 = np.random.uniform(size = 1)[0]
x2 = np.random.uniform(size = 1)[0]
x3 = np.random.uniform(size = 1)[0]
x4 = np.random.uniform(size = 1)[0]
x5 = np.random.uniform(size = 1)[0]
test = np.array([[x1, x2, x3, x4, x5]])

rep = 500
n_sub = 100

print("========================= B = 500 =========================")

pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 1000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 1000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 2500 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 2500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


print("========================= B = 5000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 5000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


  0%|          | 0/500 [00:00<?, ?it/s]

========================= B = 500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=1.7276091988131106, pvalue=0.4215551788607791)
variance ratio for uc: 6.171062426771843
C.C. for uc: 1.0
========================= B = 1000 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=0.7224429711080578, pvalue=0.6968246447814377)
variance ratio for uc: 3.697707373445286
C.C. for uc: 1.0
========================= B = 2500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=4.2958243673755305, pvalue=0.11672760935086736)
variance ratio for uc: 1.8111393479907423
C.C. for uc: 0.99
========================= B = 5000 =========================


100%|██████████| 500/500 [52:35<00:00,  6.31s/it]

normal test: NormaltestResult(statistic=0.7108379496447758, pvalue=0.7008797307231058)
variance ratio for uc: 1.3059734561476435
C.C. for uc: 0.982


In [12]:
# np.random.seed(11)

# x1 = np.random.uniform(size = 1)[0]
# x2 = np.random.uniform(size = 1)[0]
# x3 = np.random.uniform(size = 1)[0]
# x4 = np.random.uniform(size = 1)[0]
# x5 = np.random.uniform(size = 1)[0]
# test = np.array([[x1, x2, x3, x4, x5]])

rep = 500
n_sub = 250

print("========================= B = 500 =========================")

pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 1000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 1000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 2500 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 2500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


print("========================= B = 5000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 5000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


  0%|          | 0/500 [00:00<?, ?it/s]

========================= B = 500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=3.0423097796691443, pvalue=0.21845944461687533)
variance ratio for uc: 2.9888720614350857
C.C. for uc: 1.0
========================= B = 1000 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=0.3142447348108759, pvalue=0.8545994772984054)
variance ratio for uc: 1.7151975145914908
C.C. for uc: 0.986
========================= B = 2500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=2.0682902397986997, pvalue=0.3555301867294963)
variance ratio for uc: 0.8725600096030586
C.C. for uc: 0.92
========================= B = 5000 =========================


100%|██████████| 500/500 [1:10:27<00:00,  8.46s/it]

normal test: NormaltestResult(statistic=1.4629119346185995, pvalue=0.48120785689193923)
variance ratio for uc: 0.6794773378298061
C.C. for uc: 0.876


In [13]:
# np.random.seed(11)

# x1 = np.random.uniform(size = 1)[0]
# x2 = np.random.uniform(size = 1)[0]
# x3 = np.random.uniform(size = 1)[0]
# x4 = np.random.uniform(size = 1)[0]
# x5 = np.random.uniform(size = 1)[0]
# test = np.array([[x1, x2, x3, x4, x5]])

rep = 500
n_sub = 500

print("========================= B = 500 =========================")

pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 1000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 1000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 2500 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 2500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


print("========================= B = 5000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 5000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


  0%|          | 0/500 [00:00<?, ?it/s]

========================= B = 500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=1.7922666550224744, pvalue=0.4081447747230865)
variance ratio for uc: 0.00017390678115883958
C.C. for uc: 0.014
========================= B = 1000 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=3.828670428433397, pvalue=0.1474398159071016)
variance ratio for uc: 9.632212436267316e-05
C.C. for uc: 0.006
========================= B = 2500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=0.38118570498769966, pvalue=0.8264690144571273)
variance ratio for uc: 3.6361264149899795e-05
C.C. for uc: 0.006
========================= B = 5000 =========================


100%|██████████| 500/500 [1:43:54<00:00, 12.47s/it]

normal test: NormaltestResult(statistic=4.03340901454913, pvalue=0.1330933513294602)
variance ratio for uc: 1.9055614248466794e-05
C.C. for uc: 0.004


### test point 3

In [14]:
np.random.seed(13)

x1 = np.random.uniform(size = 1)[0]
x2 = np.random.uniform(size = 1)[0]
x3 = np.random.uniform(size = 1)[0]
x4 = np.random.uniform(size = 1)[0]
x5 = np.random.uniform(size = 1)[0]
test = np.array([[x1, x2, x3, x4, x5]])

rep = 500
n_sub = 100

print("========================= B = 500 =========================")

pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 1000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 1000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 2500 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 2500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


print("========================= B = 5000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 5000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


  0%|          | 0/500 [00:00<?, ?it/s]

========================= B = 500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=3.902339533193557, pvalue=0.1421077414326795)
variance ratio for uc: 4.177394574214453
C.C. for uc: 0.998
========================= B = 1000 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=0.09987261395790169, pvalue=0.9512900131059865)
variance ratio for uc: 2.759985341117035
C.C. for uc: 0.998
========================= B = 2500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=0.4679961035266911, pvalue=0.7913633576571825)
variance ratio for uc: 1.6239331229285963
C.C. for uc: 0.972
========================= B = 5000 =========================


100%|██████████| 500/500 [52:41<00:00,  6.32s/it]

normal test: NormaltestResult(statistic=2.1748782073432333, pvalue=0.33707861271855327)
variance ratio for uc: 1.1624354169590987
C.C. for uc: 0.946


In [15]:
# np.random.seed(7)

# x1 = np.random.uniform(size = 1)[0]
# x2 = np.random.uniform(size = 1)[0]
# x3 = np.random.uniform(size = 1)[0]
# x4 = np.random.uniform(size = 1)[0]
# x5 = np.random.uniform(size = 1)[0]
# test = np.array([[x1, x2, x3, x4, x5]])

rep = 500
n_sub = 250

print("========================= B = 500 =========================")

pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 1000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 1000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 2500 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 2500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


print("========================= B = 5000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 5000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


  0%|          | 0/500 [00:00<?, ?it/s]

========================= B = 500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=1.394159591740181, pvalue=0.4980375536473911)
variance ratio for uc: 2.3565567037318766
C.C. for uc: 0.99
========================= B = 1000 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=0.041813099463316926, pvalue=0.9793104771203334)
variance ratio for uc: 1.6739872302235885
C.C. for uc: 0.972
========================= B = 2500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=2.3659474253911292, pvalue=0.30636633719725637)
variance ratio for uc: 0.762794685635235
C.C. for uc: 0.896
========================= B = 5000 =========================


100%|██████████| 500/500 [1:09:46<00:00,  8.37s/it]

normal test: NormaltestResult(statistic=1.9362427241009903, pvalue=0.3797958672490879)
variance ratio for uc: 0.5756318717026367
C.C. for uc: 0.834


In [16]:
# np.random.seed(7)

# x1 = np.random.uniform(size = 1)[0]
# x2 = np.random.uniform(size = 1)[0]
# x3 = np.random.uniform(size = 1)[0]
# x4 = np.random.uniform(size = 1)[0]
# x5 = np.random.uniform(size = 1)[0]
# test = np.array([[x1, x2, x3, x4, x5]])

rep = 500
n_sub = 500

print("========================= B = 500 =========================")

pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 1000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 1000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))



print("========================= B = 2500 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 2500)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


print("========================= B = 5000 =========================")
pred = []
var_uc = []
var_c = []

for r in tqdm(range(rep)):
    
    X, y = gen(500)
    
    model = RandomForestRegressor(n_estimators = 5000)
    model.fit(X, y, n_subsamples = n_sub, replace = False)
    pred.append(model.predict(test)[0])
#     var_c.append(predict_var(model, test)[0])
    var_uc.append(predict_var(model, test, method = "IJ")[0])

print("normal test: " + str(stats.normaltest(pred)))
# print("variance ratio for c: " + str(np.mean(var_c) / np.var(pred)))
print("variance ratio for uc: " + str(np.mean(var_uc) / np.var(pred)))

target = np.mean(pred)
# count_c = 0
# for r in range(rep):
#     lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#     upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#     if target >= lower and target <= upper:
#         count_c += 1
count_uc = 0
for r in range(rep):
    lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
    upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
    if target >= lower and target <= upper:
        count_uc += 1        
# print("C.C. for c: " + str(count_c / rep))
print("C.C. for uc: " + str(count_uc / rep))


  0%|          | 0/500 [00:00<?, ?it/s]

========================= B = 500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=7.099276847840463, pvalue=0.02873502767475506)
variance ratio for uc: 0.00016132783171664226
C.C. for uc: 0.004
========================= B = 1000 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=8.520536496251461, pvalue=0.014118514637065656)
variance ratio for uc: 8.484956057184751e-05
C.C. for uc: 0.014
========================= B = 2500 =========================


  0%|          | 0/500 [00:00<?, ?it/s]

normal test: NormaltestResult(statistic=13.336765139583738, pvalue=0.001270451957474542)
variance ratio for uc: 3.4812761937803986e-05
C.C. for uc: 0.006
========================= B = 5000 =========================


100%|██████████| 500/500 [1:44:29<00:00, 12.54s/it]

normal test: NormaltestResult(statistic=10.923771663277495, pvalue=0.004245541813651881)
variance ratio for uc: 1.6525627907195875e-05
C.C. for uc: 0.006
